In [1]:
%load_ext memory_profiler
%load_ext line_profiler

In [2]:
import nimare as nim
from nimare import annotate

dset = nim.dataset.Dataset.load(
    '/Users/taylor/Documents/nbc/nimare-paper/data/neurosynth_dataset_with_abstracts.pkl.gz'
)
texts_df = dset.texts
texts_df = texts_df.reset_index(drop=False)

counts_df = annotate.text.generate_counts(
    texts_df, 
    text_column='abstract', 
    tfidf=False, 
    max_df=0.1, 
    min_df=0.05,
)
coordinates_df = dset.coordinates
counts_df = counts_df.iloc[:5000]

INFO:nimare.annotate.text:Retaining 14369/14371 studies


In [3]:
#%mprun -T gclda_init -f annotate.gclda.GCLDAModel.__init__ model = annotate.gclda.GCLDAModel(counts_df, coordinates_df, mask=dset.masker.mask_img, n_topics=50, symmetric=True, n_regions=2, seed_init=1)

In [4]:
#print(open('gclda_init', 'r').read())

In [5]:
#%mprun -T gclda -f annotate.gclda.GCLDAModel.fit model.fit(n_iters=1, loglikely_freq=1000)

In [6]:
#print(open('gclda', 'r').read())

In [7]:
model = annotate.gclda.GCLDAModel(counts_df, coordinates_df, mask=dset.masker.mask_img, n_topics=50, symmetric=True, n_regions=2, seed_init=1)
%lprun -T gclda_time -f annotate.gclda.GCLDAModel._update_peak_assignments model.fit(n_iters=50, loglikely_freq=1000)

INFO:nimare.annotate.gclda:Constructing/Initializing GCLDA Model


*** KeyboardInterrupt exception caught in code being profiled.
*** Profile printout saved to text file 'gclda_time'. *** KeyboardInterrupt exception caught in code being profiled.


In [8]:
print(open('gclda_time', 'r').read())

Timer unit: 1e-06 s

Total time: 552.325 s
File: /Users/taylor/Documents/tsalo/NiMARE/nimare/annotate/gclda.py
Function: _update at line 379

Line #      Hits         Time  Per Hit   % Time  Line Contents
   379                                               def _update(self, loglikely_freq=1):
   380                                                   """Run a complete update cycle (sample z, sample y&r, update regions).
   381                                           
   382                                                   Parameters
   383                                                   ----------
   384                                                   loglikely_freq : :obj:`int`, optional
   385                                                       The frequency with which log-likelihood is updated. Default value
   386                                                       is 1 (log-likelihood is updated every iteration).
   387                                                   "

In [9]:
import numpy as np
p = np.random.random(100)
p[5] = 1000
p = p / np.sum(p)
idx = np.random.choice(a=np.arange(100), size=1, p=p)[0]
idx2 = np.where(np.random.multinomial(1, p))[0][0]
print(idx)
print(idx2)
p_peak_g_topic = np.random.random(10000)

5
5


In [10]:
%timeit -n 1000 np.random.choice(a=np.arange(100), size=1, p=p)[0]

32.4 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit -n 1000 np.where(np.random.multinomial(1, p))[0][0]

15.7 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit np.tile(p_peak_g_topic, (2, 1))

8.85 µs ± 290 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit np.vstack((p_peak_g_topic, p_peak_g_topic))

9.66 µs ± 528 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
%timeit np.array([p_peak_g_topic] * 2)

7.05 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
np.array([p_peak_g_topic] * 2)

array([[0.41871426, 0.36844783, 0.77029884, ..., 0.4048586 , 0.74305738,
        0.02430984],
       [0.41871426, 0.36844783, 0.77029884, ..., 0.4048586 , 0.74305738,
        0.02430984]])